# Weather Data Analysis
*Author: Olga Sieradzan*

This notebook focuses on analying and forcating weather patterns using the Weather data set from kaggle. 

***

The notebook is organized as follows:

**Part 1**: Data Cleaning

**Part 2**: Exploratory data analysys

**Part 3**: Weather forecasting based on historical data

***


## Data Cleaning

This part consist of choing usefull columns, converting data types , handling missing values or Nan's, 

***

### Libraries and option setting

In [1]:
import pandas as pd
import numpy as np
import sys
import re

# Setting up max display in notebook 
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

***

## Raw data 

Dataset consist of **14 colums** describing weather at the small town in Poland, collected by the meterodical station. Data was collected since 1980 till 2024 hourly, maiking the dataset **389497 rows** big

The variables in dataset are: 

* Temperature
* Relative humidity
* Dew point
* Precipitation (mm)
* Rain (mm)
* Snowfall (mm)
* Pressure msl (hPa)
* Surface_pressure (hPa)
* Cloud cover (%)
* Cloud cover low (%)
* Cloud cover mid (%)
* Cloud cover high (%)
* Vapour pressure deficit (kPa)
* Wind speed (km/h)





In [3]:
raw_data = pd.read_csv("C:/Users/olgas/OneDrive/Documents/GitHub/Time-Series-Weather-Prediction-Notebook/Data/Weather_Data_1980_2024(hourly).csv", parse_dates = ['time'])
raw_data.head()

,time,temperature,relative_humidity,dew_point,precipitation (mm),rain (mm),snowfall (cm),pressure_msl (hPa),surface_pressure (hPa),cloud_cover (%),cloud_cover_low (%),cloud_cover_mid (%),cloud_cover_high (%),vapour_pressure_deficit (kPa),wind_speed_10m (km/h)
0,1980-01-01 00:00:00,12.7,83,10.0,0.0,0.0,0,1012.8,945.1,1,1,0,0,0.25,7.5
1,1980-01-01 01:00:00,12.9,82,9.9,0.0,0.0,0,1012.2,944.5,4,4,0,0,0.26,7.9
2,1980-01-01 02:00:00,13.2,82,10.2,0.0,0.0,0,1012.3,944.7,13,14,0,0,0.27,7.5
3,1980-01-01 03:00:00,15.9,78,12.0,0.0,0.0,0,1013.5,946.4,23,26,0,0,0.40,6.6
4,1980-01-01 04:00:00,19.4,67,13.1,0.0,0.0,0,1014.6,948.2,9,10,0,0,0.75,8.7


In [ ]:
raw_data.dtypes

time                             datetime64[ns]
temperature                             float64
relative_humidity                         int64
dew_point                               float64
precipitation (mm)                      float64
rain (mm)                               float64
snowfall (cm)                             int64
pressure_msl (hPa)                      float64
surface_pressure (hPa)                  float64
cloud_cover (%)                           int64
cloud_cover_low (%)                       int64
cloud_cover_mid (%)                       int64
cloud_cover_high (%)                      int64
vapour_pressure_deficit (kPa)           float64
wind_speed_10m (km/h)                   float64
dtype: object

: 

As the table above show, there will be no problem with data types since all of the data is either intiger or float. 